# Assignment - 10 Description

In Cirq implement the Deutsch-Jozsa algorithm and the Bernstein-Vazirani algorithm.  Write detailed comments in the code about why it works.  Run the programs on the simulator.  Write a report that covers the following three points.

1. Design

    * Present the design of how you implemented the black-box function U_f.  Assess how visually neat and easy to read it is.
    * Present the design for how you prevent the user of U_f from accessing the implementation of U_f.  Assess how well you succeeded.
    * Present the design of how you parameterized the solution in n.
    * Discuss the number of lines and percentage of code that your two programs share.  Assess how well you succeeded in reusing code from one program to the next.


2. Evaluation

    * Discuss your effort to test the two programs and present results from the testing.  Discuss whether different cases of U_f lead to different execution times.
    * What is your experience with scalability as n grows?  Present a diagram that maps n to execution time.

3. Instructions

    * Present a README file that describes how to input the function f, how to run the program, and how to understand the output.
    * Submit three files, one for each program and one with the report.

# Bernstein- Vazirani: The Problem

1. Input: a function f: {0,1}^n → {0,1}.
2. Assumption: f(x) = a * x + b.
3. Output: a, b.

Notation: {0,1}^n is the set of bit strings of length n, a is an unknown bit string of length n, * is inner product mod 2, + is addition mod 2, and b is an unknown single bit.

# Headers Required

In [1]:
import numpy as np
import time
import itertools as itert
import cirq

# Defining our current function


#### This is a random function generator which gives a, b for current running instance.

#### This helps ensure that our test cases are not biased

In [2]:
def function_generator(n):
    # a- binary string of length n
    a = np.random.randint(low=0, high=2, size=n)
    print ("This is the (randomly chosen) value of a: ", a)
    # b- single bit binary digit
    b = np.random.randint(low=0, high=2, size=1)
    print ("This is the (randomly chosen) value of b: ", b)
    return a,b

# Solving for 'b' classically 

#### As per class notes, Week 3 -> Palsberg; Algorithms: Bernstein-Vazirani File, we solve for 'b' classically and 'a' using quantum computing

In [3]:

def solveB (n,a,b):
    zero_vec=[0]*n
    curr_dot_product=0
    for i in range(0,n):
        curr_dot_product += a[i] * zero_vec[i]
    curr_dot_product %= 2
    curr_dot_product+=b
    curr_dot_product %= 2
    print("On solving b classically we get, b = ", curr_dot_product)

# Creating the Uf Matrix

#### The code differs from the Deutsch Jozsa algorithm only in terms of parameters and function application.
#### Using class to block user access to Uf and treat Uf as blackbox oracle 

In [4]:
class Uf():
    
        
    #this function creates the blackbox oracle Uf matrix for a given function f that is parameterised for input size n
        
    
    def __blackboxUf(self,n,a,b):
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0
        Uf = np.zeros((2**(n+1),2**(n+1)))

        bfx = 0

        # the for loops given below form the different pattern combinations that we need. The mapping between these combination and Uf matrix indices is stored in indices_dict
        for i in range(2**(n+1)):
            binary_form = ('{0:0'+str(n+1)+'b}').format(i) 
            indices_dict[binary_form] = counter
            counter += 1

        #print(indices_dict)
        #we then iterate through this pattern dictionary
        for key,val in indices_dict.items():
        
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = key[0:n]
            #print("X= ", x)
            curr_dot_product = 0
            #for k, expand_q in enumerate(x):
                #print(x)
                #print(k)
                #curr_dot_product += a[k] * int(expand_q)
            for i in range(0,n):
                curr_dot_product += a[i] * int(x[i])
            curr_dot_product %= 2
            curr_dot_product= int(curr_dot_product) + int(b)
            curr_dot_product %= 2
        
            # f from current string
        
            fx= str(curr_dot_product) 
        
            #fx represents the output of function f given the input x
            #fx = str(f(x))
        
            #b1 is the last bit of the bit pattern in the dictionary item
            b1 = key[n]
        
            #below we have the (f(x) + b) mod 2
            if(b1==fx):
                bfx = '0'
            else:
                bfx = '1'
                # print(bfx)
        
            #the final bit string is the concatenation of the input x and bfx
            target = x + bfx
            # print('t',target)
        
            #using indices_dict we can now find the index that corresponds to this output
            target_index = indices_dict[target]
            # print(val,target_index)
        
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[val][target_index] = 1
        
        return Uf
    def createUf(self,n,a,b):
        return self.__blackboxUf(n,a,b)

# Creating Gate out of Uf Matrix

In [5]:
class Oracle(cirq.Gate):
    def __init__(self, n, UfMatrix, name):
        self.__n=n
        self.__UfMatrix=UfMatrix
        self.__name=name
    def num_qubits(self):
        return self.__n 
    def _unitary_(self):
        #return self.__UfMatrix
        #print("Unitary array", np.squeeze(np.asarray(self.__UfMatrix)))
        return np.squeeze(np.asarray(self.__UfMatrix))
        
    def __str__(self):
        return self.__name
    

# Creating our main circuit

* This circuit will be used to find 'a'. [Displayed in Results]

* Implementation similar to Deutsch Jozsa except in terms of params and some function calls.

* After measurement, the last bit can be discarded as its the result from the helper bit. All other bits denote 'a'

* Circuit should look like

<img src="circuit.png" alt="Circuit" style="width: 300px; float: left"/>




In [6]:
#def runMainCircuit(function,n,nTrials):
def runMainCircuit():
    
    
    # Taking the input n from the user
    n= int(input("Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: "))
    
    # one extra bit as our helper bit 
    n = n+1
    
    # creating an instance of Uf
    
    uf = Uf()
    # defining start of Cirq circuit
    
    c=cirq.Circuit()
    
    
    
    # Time taken by program needs to be checked.
    start = time. time()
    
    # generating my function
    print ("\n\n-----------This function is not accessible to user-----------")
    print ("Function generator output shown for verification/ proof of correctness ")
    
    curr_a, curr_b = function_generator(n)
    print ("-----------Restricted section over-----------\n\n")
    #classically solving b
    
    solveB(n-1,curr_a,curr_b)
    
    
    qubits = cirq.LineQubit.range(n)
    # setting last qubit to 1
    
    
    c.append(cirq.X(qubits[n-1]))
    
    # adding Hadamard gates to all qubits
    
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])
        
    # creating our Uf matrix 
    UfMatrix = uf.createUf(n-1,curr_a,curr_b) 
    
    # creating Uf gate
    
    uf_bv= Oracle(n, UfMatrix, "UF_BV")
    
    
    
    # adding Uf gate
    
    
    c.append(uf_bv(*qubits))
    
    # helper bit does not require H gate. Result is treated as trash/ garbage
    
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])    
    
    
    # measurements
    for i in range(0,n):
        c.append(cirq.measure(qubits[i])) 
        
    print(c)
    
    simulator = cirq.Simulator()
    
    result = simulator.run(c, repetitions=1)
    
    print("Results: ")
    print(result)
    
    end = time. time()
    print("Note: last bit is trashed as it is output of helper bit")
    print("Time taken by program: ", end-start)

# Bernstein-Vazirani - Test Case Examples

## Trial 1: n =1

In [7]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 1


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 1]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
0: ───H───────UF_BV───H───M───
              │
1: ───X───H───#2──────H───M───
Results: 
0=1
1=1
Note: last bit is trashed as it is output of helper bit
Time taken by program:  0.019974946975708008


## Trial 2: n=2

In [8]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 2


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 0 0]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
0: ───H───────UF_BV───H───M───
              │
1: ───H───────#2──────H───M───
              │
2: ───X───H───#3──────H───M───
Results: 
0=1
1=0
2=1
Note: last bit is trashed as it is output of helper bit
Time taken by program:  0.01997661590576172


## Trial 3: n=3

In [9]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 3


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 0 0 1]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
0: ───H───────UF_BV───H───M───
              │
1: ───H───────#2──────H───M───
              │
2: ───H───────#3──────H───M───
              │
3: ───X───H───#4──────H───M───
Results: 
0=1
1=0
2=0
3=1
Note: last bit is trashed as it is output of helper bit
Time taken by program:  0.027962207794189453


## Trial 4: n=4

In [10]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 4


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [0 1 0 0 1]
This is the (randomly chosen) value of b:  [1]
-----------Restricted section over-----------


On solving b classically we get, b =  [1]
0: ───H───────UF_BV───H───M───
              │
1: ───H───────#2──────H───M───
              │
2: ───H───────#3──────H───M───
              │
3: ───H───────#4──────H───M───
              │
4: ───X───H───#5──────H───M───
Results: 
0=0
1=1
2=0
3=0
4=1
Note: last bit is trashed as it is output of helper bit
Time taken by program:  0.03595232963562012
